In [12]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

# ——————————————————————————————————
# 1) 데이터 로드
# ——————————————————————————————————
# TS (TimeSformer) 임베딩
X_ts_train = np.load('embbeding_data/timesformer/train/embeddings.npy')
y_train     = np.load('embbeding_data/timesformer/train/labels.npy').ravel()
X_ts_test  = np.load('embbeding_data/timesformer/test/embeddings.npy')
y_test     = np.load('embbeding_data/timesformer/test/labels.npy').ravel()

# ST (ST-GCN) 임베딩 (3D → 2D flatten)
X_st_train = np.load('embbeding_data/stgcnpp/train/embeddings.npy')
X_st_test  = np.load('embbeding_data/stgcnpp/test/embeddings.npy')
# 세그먼트 축(1)을 평균 처리
X_st_train = X_st_train.mean(axis=1)
X_st_test  = X_st_test.mean(axis=1)

# ——————————————————————————————————
# 2) TS-only 성능
# ——————————————————————————————————
clf_ts = LogisticRegression(max_iter=1000).fit(X_ts_train, y_train)
pred_ts_train = clf_ts.predict(X_ts_train)
prob_ts_train = clf_ts.predict_proba(X_ts_train)[:,1]
pred_ts_test  = clf_ts.predict(X_ts_test)
prob_ts_test  = clf_ts.predict_proba(X_ts_test)[:,1]

print("TS-only Train  acc:", accuracy_score(y_train, pred_ts_train))
print("TS-only Train  AUC:", roc_auc_score(y_train, prob_ts_train))
print("TS-only Test   acc:", accuracy_score(y_test,  pred_ts_test))
print("TS-only Test   AUC:", roc_auc_score(y_test,  prob_ts_test))

# ——————————————————————————————————
# 3) ST-only 성능
# ——————————————————————————————————
clf_st = LogisticRegression(max_iter=1000).fit(X_st_train, y_train)
pred_st_train = clf_st.predict(X_st_train)
prob_st_train = clf_st.predict_proba(X_st_train)[:,1]
pred_st_test  = clf_st.predict(X_st_test)
prob_st_test  = clf_st.predict_proba(X_st_test)[:,1]

print("\nST-only Train acc:", accuracy_score(y_train, pred_st_train))
print("ST-only Train AUC:", roc_auc_score(y_train, prob_st_train))
print("ST-only Test  acc:", accuracy_score(y_test,  pred_st_test))
print("ST-only Test  AUC:", roc_auc_score(y_test,  prob_st_test))

# ——————————————————————————————————
# 4) Fusion 성능 (두 임베딩 concatenate)
# ——————————————————————————————————
X_f_train = np.hstack([X_ts_train, X_st_train])
X_f_test  = np.hstack([X_ts_test,  X_st_test])
clf_f = LogisticRegression(max_iter=1000).fit(X_f_train, y_train)
pred_f_train = clf_f.predict(X_f_train)
prob_f_train = clf_f.predict_proba(X_f_train)[:,1]
pred_f_test  = clf_f.predict(X_f_test)
prob_f_test  = clf_f.predict_proba(X_f_test)[:,1]

print("\nFusion Train acc:", accuracy_score(y_train, pred_f_train))
print("Fusion Train AUC:", roc_auc_score(y_train, prob_f_train))
print("Fusion Test  acc:", accuracy_score(y_test,  pred_f_test))
print("Fusion Test  AUC:", roc_auc_score(y_test,  prob_f_test))


TS-only Train  acc: 1.0
TS-only Train  AUC: 1.0
TS-only Test   acc: 0.4092526690391459
TS-only Test   AUC: 0.4127128953771289

ST-only Train acc: 0.6982758620689655
ST-only Train AUC: 0.7400198281559816
ST-only Test  acc: 0.5231316725978647
ST-only Test  AUC: 0.4989355231143553

Fusion Train acc: 1.0
Fusion Train AUC: 1.0
Fusion Test  acc: 0.4199288256227758
Fusion Test  AUC: 0.40379156528791565
